# Use FairMedFM to perform CLIP Zero-shot and Adaptation

# Install FairMedFM

In [1]:
!git clone https://github.com/FairMedFM/FairMedFM

Cloning into 'FairMedFM'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (65/65), done.
remote: Total 418 (delta 34), reused 44 (delta 22), pack-reused 330
Receiving objects: 100% (418/418), 2.99 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [2]:
%cd FairMedFM

/content/FairMedFM


We use PAPILA as an example for demonstration.

In [3]:
%mkdir data
%cd data
!wget https://object-arbutus.cloud.computecanada.ca/rjin/PAPILA.zip
!unzip PAPILA.zip
%cd ..

/content/FairMedFM/data
--2024-07-31 19:36:08--  https://object-arbutus.cloud.computecanada.ca/rjin/PAPILA.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 589491638 (562M) [application/zip]
Saving to: ‘PAPILA.zip’

PAPILA.zip          100%[===================>] 562.18M  98.5MB/s    in 6.1s    

2024-07-31 19:36:14 (91.6 MB/s) - ‘PAPILA.zip’ saved [589491638/589491638]

Archive:  PAPILA.zip
   creating: PAPILA/
   creating: PAPILA/split/
  inflating: PAPILA/split/new_train.csv  
  inflating: PAPILA/split/new_test.csv  
   creating: PAPILA/data/
  inflating: PAPILA/data/.DS_Store   
   creating: PAPILA/data/ClinicalData/
  inflating: PAPILA/data/ClinicalData/patient_data_od.xlsx  
  inflating: PAPILA/data/ClinicalData/patient_data_os.xlsx  
   creatin

In [4]:
!wget https://object-arbutus.cloud.computecanada.ca:443/rjin/FairMedFM/pretrained.zip
!unzip pretrained.zip
!rm -f pretrained.zip

--2024-07-31 19:36:24--  https://object-arbutus.cloud.computecanada.ca/rjin/pretrained.zip
Resolving object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)... 206.12.48.146
Connecting to object-arbutus.cloud.computecanada.ca (object-arbutus.cloud.computecanada.ca)|206.12.48.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6263457190 (5.8G) [application/zip]
Saving to: ‘pretrained.zip’

pretrained.zip      100%[===================>]   5.83G  89.5MB/s    in 73s     

2024-07-31 19:37:37 (82.0 MB/s) - ‘pretrained.zip’ saved [6263457190/6263457190]

Archive:  pretrained.zip
   creating: pretrained/
   creating: pretrained/medclip-resnet/
  inflating: pretrained/medclip-resnet/pytorch_model.bin  
 extracting: pretrained/medclip-resnet/medclip-pretrained.zip  
   creating: pretrained/C2L/
  inflating: pretrained/C2L/res18_C2L.pth  
   creating: pretrained/mococxr/
  inflating: pretrained/mococxr/r8w-00001-v2.pth.tar  
   creating: pretraine

In [5]:
!pip install icecream==2.1.3
!pip install SimpleITK==2.3.1
!pip install einops==0.8.0
!pip install ipdb==0.13.13
!pip install git+https://github.com/facebookresearch/segment-anything.git@6fdee8f2727f4506cfbbe553e23b895e2795658
!pip install timm==1.0.3
!pip install open-clip-torch==2.24.0
!pip install salesforce-lavis==1.0.2
!pip install git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
!pip install MedCLIP==0.0.3
!pip install torchmetrics==1.4.0.post0
!pip install peft==0.11.1
!pip install git+https://github.com/transcendentsky/tutils.git@8b02b20c9caa5553c96a6413b739ac47385fd730
!pip install transformers==4.30.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 33.5 MB/s eta 0:00:00
  Cloning https://github.com/facebookresearch/segment-anything.git (to revision 6fdee8f2727f4506cfbbe553e23b895e2795658) to /tmp/pip-req-build-aym0earn
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-aym0earn
  Running command git checkout -q 6fdee8f2727f4506cfbbe553e23b895e2795658
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 6fdee8f2727f4506cfbbe553e23b895e2795658
  Preparing metadata (setup.py) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36591 sha256=d9b17f4d8434f176f47dea889616a507611ecb832f4fd97ce8a6acb047e12ec1
  Stored in directory: /tmp/pip-ephem-wheel-cache-krx76wqh/wheels/ed/f9/1

  Cloning https://github.com/openai/CLIP.git (to revision a1d071733d7111c9c014f024669f959182114e33) to /tmp/pip-req-build-j9emlfda
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-j9emlfda
  Running command git rev-parse -q --verify 'sha^a1d071733d7111c9c014f024669f959182114e33'
  Running command git fetch -q https://github.com/openai/CLIP.git a1d071733d7111c9c014f024669f959182114e33
  Running command git checkout -q a1d071733d7111c9c014f024669f959182114e33
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369500 sha256=0afb7ec6fd43c4c94c91bdff8ffa4d51b1a467295478734252a8a5d8e32514bf
  Stored in directory: /root/.cache/pip/wheels/df/64/50/148b28b28a0f816d0e876207653618a4ec90e3a9c0f9c6e2a5
Successfully built clip
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━

In [6]:
!pip freeze

absl-py==1.4.0
accelerate==0.32.1
aiohttp==3.9.5
aiosignal==1.3.1
alabaster==0.7.16
albucore==0.0.12
albumentations==1.4.12
altair==4.2.2
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==3.7.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.5.1
arviz==0.18.0
asn1crypto==1.5.1
astropy==6.1.2
astropy-iers-data==0.2024.7.29.0.32.7
asttokens==2.4.1
astunparse==1.6.3
async-timeout==4.0.3
atpublic==4.1.0
attrs==23.2.0
audioread==3.0.1
autograd==1.6.2
Babel==2.15.0
backcall==0.2.0
beautifulsoup4==4.12.3
bidict==0.23.1
bigframes==1.11.1
bleach==6.1.0
blinker==1.4
blis==0.7.11
blosc2==2.0.0
bokeh==3.4.3
bqplot==0.12.43
braceexpand==0.1.7
branca==0.7.2
build==1.2.1
CacheControl==0.14.0
cachetools==5.4.0
catalogue==2.0.10
certifi==2024.7.4
cffi==1.16.0
cfgv==3.4.0
chardet==3.0.4
charset-normalizer==3.3.2
chex==0.1.86
clarabel==0.9.0
click==8.1.7
click-plugins==1.1.1
cligj==0.7.2
clip @ git+https://github.com/openai/CLIP.git@a1d071733d7111c9c014f024669f959182114e33
clou

# CLIP-ZS Example

Many medical FMs are trained based on CLIP with image-text pairs, e.g., MedCLIP. CLIP is known for its awesome zeroshot ability. Here, we first use naive MedCLIP to perform zeroshot on one dataset.

In [7]:
!CUDA_VISIBLE_DEVICES=0 python main.py --task cls --usage clip-zs --dataset PAPILA --model MedCLIP --sensitive_name Sex

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam_builder.build_tinysam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_11m

As we can see, the performance is not good. This is because MedCLIP is pretrained on X-rays but not retina images. Let's try using CLIP-Adaptor to adjust it.

# CLIP-Adaptor Example

Many medical FMs are trained based on CLIP with image-text pairs, e.g., MedCLIP. CLIP is known for its awesome zeroshot ability. Here, we first use naive MedCLIP to perform zeroshot on one dataset.

In [8]:
!CUDA_VISIBLE_DEVICES=0 python main.py --task cls --usage clip-adapt --dataset PAPILA --model MedCLIP --sensitive_name Sex --method erm --total_epochs 50 --warmup_epochs 5 --blr 2.5e-2 --batch_size 128 --optimizer adamw --min_lr 1e-5 --weight_decay 0.05

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam_builder.build_tinysam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/content/FairMedFM/models/sam_builder/build_tinysam.py:819: UserWarning: Overwriting tiny_vit_11m

We adapt CLIP for 50 epochs, the AUC grows to 0.23. However, the loss is still not converging and we can get better results if we do more epochs.